In [2]:
import requests
import os
import sys
import pandas as pd
import numpy as np

# import key sklearn ml algos
# import key sklearn metrics
# import 

from typing import Optional
import glob
from scipy import stats
# Add the parent directory of this notebook to sys.path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

from project_tools import project_utils, project_class

import datetime
import json
from tqdm.notebook import tqdm
import gc
# import ds_utils
import random
import matplotlib.pyplot as plt
%matplotlib inline

from importlib import reload
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# Or for more precise control
pd.set_option('display.float_format', '{:.0f}'.format)


# notes

In [3]:
# check package - lightgbm, xgboost, bayesian-optimization, 

In [4]:
# major technical items to review:
# 1. groupby-aggregator handy func
# 2. numerai validator class with different algo
# 3. null hypo. feature selection routine - if not using the numerai class
# 4. weight optimisation routine 

In [5]:
# handy code to develop to prep for interview
# 1. final position to target transformation 
# 2. probability normalization for horses in the same race
# 3. prediction optimisation for metric per race 

In [6]:
# generic plan 
# 1. set up package and load data
# 2. initial analysis
# 2.1 dataframe.info 
# 2.2 key columns format, null count
# 2.3 remove unrelevant data - check finishing position, win_odds, missing finishing time, etc
# 2.4 identify data cleaning target
# 3. target analysis    
# 4. generate machine trainable dataset - for tree and for linear & NN algo.


# 5. generate measurable targets - if needed 
# 5.1 identify "very import features" - do transformation of those if needed 
# 6. generate additional features
# 7. 


# to-do:
# 1. create generate data analysis coding blocks including simple visualizations on notebook
# 2. switch to jupyber lab setting, and write the code there 

In [7]:
# jockey performance
# training performance

In [8]:
# domain specific feature:
# horse group dynamics
# jockey group dynamics
# horse/trainer/jockey/race course dynamics
# horse weather dynamcs

In [9]:
horse_result_file  = '../data/race-result-horse.csv'
race_result_file = '../data/race-result-race.csv'

horse_df = pd.read_csv(horse_result_file)
race_df = pd.read_csv(race_result_file)

horse_df['finishing_position'] = horse_df['finishing_position'].fillna('NA')

# data cleaning

In [10]:
def text_to_numeric(text):
    """
    Convert text to numeric (int or float), return np.nan if not possible
    
    Args:
        text: Input text/value to convert
        
    Returns:
        int, float or np.nan: Converted numeric value or np.nan if conversion fails
    """
    # Handle None, empty string, or np.nan input
    if text is None or text == '' or (isinstance(text, float) and np.isnan(text)):
        return np.nan
    
    # If input is already numeric, return as is
    if isinstance(text, (int, float)):
        return text
    
    # Convert to string and strip whitespace
    text = str(text).strip()
    
    try:
        # Try integer first
        value = int(text)
        return value
    except ValueError:
        try:
            # Try float
            value = float(text)
            return value
        except ValueError:
            return np.nan

In [11]:
def convert_length_behind(text):
    """
    Convert length behind text to numerical value
    
    Args:
        text (str): Text representation of length behind
        
    Returns:
        float: Numerical value of length behind
    """
    # Special case mappings
    special_cases = {
        '-': 0,
        'N': 0.3,
        'SH': 0.1,
        'HD': 0.25,
        'NOSE': 0.05,
        '+NOSE': 0.75,
        'ML': 50,
        '+SH': 0.15,
        'TO': 0.1,
        '+1/2': 99,
        '---': 99
    }
    
    # Check if input is in special cases
    if text in special_cases:
        return special_cases[text]
    
    try:
        # Try to convert direct number first
        return float(text)
    except ValueError:
        # Handle compound fractions (e.g., "4-1/4", "5-1/2")
        parts = text.split('-')
        
        if len(parts) == 1:
            # Handle pure fractions (e.g., "1/4", "1/2")
            if '/' in parts[0]:
                num, denom = map(float, parts[0].split('/'))
                return num/denom
        else:
            # Handle whole number with fraction
            whole = float(parts[0])
            if '/' in parts[1]:
                num, denom = map(float, parts[1].split('/'))
                return whole + num/denom
            
        raise ValueError(f"Unable to parse length: {text}")

# Example usage:
# lengths = ['2', '4-1/4', '5-1/2', 'NOSE', '1/2', 'N']
# numeric_lengths = [convert_length_behind(x) for x in lengths]

In [12]:
def convert_timestamp_to_seconds(timestamp: str) -> float:
    """
    Convert text timestamp in format "M.SS.ss" to total seconds
    where M=minutes, SS=seconds, ss=decimal seconds
    
    Examples:
        "1.41.91" -> 101.91 (1 min 41.91 sec)
        "1.40.12" -> 100.12 (1 min 40.12 sec)
        "0.58.41" -> 58.41 (58.41 sec)
    
    Args:
        timestamp: String timestamp in M.SS.ss format
        
    Returns:
        Float value representing total seconds
    """
    parts = timestamp.split('.')
    
    if len(parts) != 3:
        raise ValueError(f"Invalid timestamp format: {timestamp}. Expected format: M.SS.ss")
        
    minutes = float(parts[0])
    seconds = float(parts[1])
    decimal = float(parts[2]) / 100  # Convert decimal part to fraction
    
    total_seconds = minutes * 60 + seconds + decimal
    
    return total_seconds

In [13]:
horse_df['clean_actual_weight'] = horse_df['actual_weight'].apply(lambda x: text_to_numeric(x))
horse_df['clean_declared_horse_weight'] = horse_df['declared_horse_weight'].apply(lambda x: text_to_numeric(x))

In [14]:
horse_df['clean_length_behind_winner'] = horse_df['length_behind_winner'].apply(lambda x: convert_length_behind(x))

In [15]:
### clean up invalid data

print(horse_df.shape)
horse_df = horse_df[horse_df['win_odds']!='---']
print(horse_df.shape)
horse_df = horse_df[horse_df['finish_time']!='---']

horse_df.shape

(30189, 22)
(29598, 22)


(29520, 22)

In [16]:
horse_df['clean_finish_time'] = horse_df['finish_time'].apply(lambda x: convert_timestamp_to_seconds(x))

In [19]:
horse_df['draw'] = horse_df['draw'].astype(np.uint8)

In [20]:
horse_df['clean_win_odds'] = horse_df['win_odds'].astype(np.float16)

In [21]:
race_df['race_course_track'] = race_df['race_course'] + '_' + race_df['track']

In [22]:
# target generation

In [23]:
clean_position = []
for i,row in horse_df.iterrows():
    row_finishing_position = row['finishing_position']
    if 'DH' in row_finishing_position:
        row_position = int(row_finishing_position[0])
    else:
        if str.isdigit(row_finishing_position):
            row_position = int(row_finishing_position)
        else:
            row_position = 99
    clean_position.append(row_position)
horse_df['clean_position'] = clean_position

In [24]:
horse_df[horse_df['clean_position']==99].shape

(1, 25)

In [25]:
horse_df['is_winner'] = (horse_df['clean_position'] == 1).astype(int)
horse_df['is_top3'] = (horse_df['clean_position'] <= 3).astype(int)


# race data clean up

In [26]:
def convert_date_to_int(date_str: str) -> int:
    """
    Convert date string in YYYY-MM-DD format to integer that preserves ordering
    
    Args:
        date_str: Date string in YYYY-MM-DD format
        
    Returns:
        Integer in format YYYYMMDD
        
    Example:
        '2015-11-18' -> 20151118
        '2016-03-31' -> 20160331
    """
    # Remove hyphens and convert to integer
    return int(date_str.replace('-', ''))

In [27]:
race_df['clean_race_date'] = race_df['race_date'].apply(lambda x:convert_date_to_int(x))

# merge data

In [28]:
horse_race_df = horse_df.merge(right=race_df, on=['race_id'], how='left')
horse_race_df.shape

(29520, 40)

In [29]:
print(horse_df.shape, race_df.shape)

(29520, 27) (2367, 14)


# feature generation

In [30]:
# feature note:
# race_course_track: the combination of race_course and track
# race course/track feature - sectional time statistics
# race cource - race course_track conditional ratio
# 
# horse feature -  ratio between actual weight, declared weight
# horse feature - length behind winner statistics
# horse feature - running position statistics
# horse feature - jockey statistics - avg finishing position (<99)
# horse feature - trainer statistic - avg finishing position (<99)
# horse feature - failed to finish ration - num <99 / num_races
 
# horse race feature - finish time/course length
# horse race feature - 



# ask perplexity for features engineering idea

# race course features

In [31]:
# Pattern classification
def classify_race_pattern(times):
    times = times.astype(float)
    diffs = np.diff(times)
    if all(diffs < 0):
        return 'progressive_speedup'
    elif all(diffs > 0):
        return 'progressive_slowdown'
    elif diffs[0] < 0 and diffs[-1] > 0:
        return 'middle_burst'
    elif diffs[0] > 0 and diffs[-1] < 0:
        return 'strong_finish'
    return 'mixed'

In [32]:
race_course_features = []
for i,row in tqdm(race_df.iterrows(), total=len(race_df)):
    stime_items = np.array(row['sectional_time'].split(' ')).astype(float)    
    item_dict = dict()
    item_dict['race_id'] = row['race_id']
    item_dict['stime_min'] = stime_items.min()
    item_dict['stime_max'] = stime_items.max()
    item_dict['stime_mean'] = stime_items.mean()
    item_dict['stime_std'] = stime_items.std()
    stime_diffs = np.diff(stime_items)
    item_dict['stime_diffs'] = stime_diffs
    item_dict['stime_pace_variance'] = np.var(stime_diffs)
    item_dict['stime_max_pace_change'] = np.abs(stime_diffs).max()
    item_dict['stime_skewness'] = stats.skew(stime_items)
    
    item_dict['stime_first_half_avg'] = np.mean(stime_items[:len(stime_items)//2].astype(float))
    item_dict['stime_second_half_avg'] = np.mean(stime_items[len(stime_items)//2:].astype(float))
    item_dict['stime_speed_bias'] = item_dict['stime_second_half_avg'] - item_dict['stime_first_half_avg']  # Negative means faster finish

    # Percentile-based features
    item_dict['stime_percentiles'] = np.percentile(stime_items.astype(float), [25, 75])
    item_dict['stime_iqr'] = item_dict['stime_percentiles'][1] - item_dict['stime_percentiles'][0]

    # Section comparisons
    item_dict['stime_fastest_section_idx'] = np.argmin(stime_items.astype(float))
    item_dict['stime_slowest_section_idx'] = np.argmax(stime_items.astype(float))


    # Trend analysis
    item_dict['is_getting_faster'] = all(stime_diffs < 0)  # True if consistently speeding up
    item_dict['is_getting_slower'] = all(stime_diffs > 0)  # True if consistently slowing down

    item_dict['race_pattern'] = classify_race_pattern(stime_items)
    race_course_features.append(item_dict)    
# break
# stime_

rc_features = pd.DataFrame.from_dict(race_course_features)



  0%|          | 0/2367 [00:00<?, ?it/s]

In [33]:
rc_features.shape

(2367, 19)

In [34]:
rc_features.sample(5)

,race_id,stime_min,stime_max,stime_mean,stime_std,stime_diffs,stime_pace_variance,stime_max_pace_change,stime_skewness,stime_first_half_avg,stime_second_half_avg,stime_speed_bias,stime_percentiles,stime_iqr,stime_fastest_section_idx,stime_slowest_section_idx,is_getting_faster,is_getting_slower,race_pattern
923,2015-146,24,28,25,2,"[-3.969999999999999, 0.28999999999999915, 0.1999999999999993]",4,4,1,26,24,-2,"[24.307499999999997, 25.45]",1,1,0,False,False,middle_burst
318,2014-319,23,24,23,0,"[-0.5, 0.00999999999999801]",0,0,1,24,23,-0,"[23.305, 23.555]",0,1,0,False,False,middle_burst
2065,2016-505,24,26,25,1,"[-1.4499999999999993, -0.21999999999999886, 0.030000000000001137, 0.3099999999999987]",0,1,1,25,24,-1,"[24.26, 24.57]",0,2,0,False,False,middle_burst
252,2014-252,14,24,21,4,"[8.129999999999999, 2.09, -0.4399999999999977]",13,8,-1,18,24,6,"[19.8275, 23.62]",4,0,2,False,False,strong_finish
191,2014-192,23,23,23,0,"[-0.08999999999999986, -0.16999999999999815]",0,0,-0,23,23,-0,"[23.295, 23.424999999999997]",0,2,0,True,False,progressive_speedup


In [35]:
rc_features.to_parquet('../feature_data/race_course_features.parquet')

In [36]:
def calculate_track_condition_ratios(df):
    # Get all unique track conditions
    res_df = df[['race_id','race_course_track']].copy()
    track_conditions = df['track_condition'].unique()
    
    # Group by race course track and calculate ratios for each condition
    track_counts = df.groupby('race_course_track').size()
    
    # Calculate ratio for each track condition
    for condition in track_conditions:
        condition_counts = df[df['track_condition'] == condition].groupby('race_course_track').size()
        ratio_col_name = f'{condition}_ratio'
        res_df[ratio_col_name] = df['race_course_track'].map(condition_counts / track_counts)
        
    # Fill NaN values with 0 (for tracks that don't have a particular condition)
    condition_ratio_cols = [f'{c}_ratio' for c in track_conditions]
    res_df[condition_ratio_cols] = res_df[condition_ratio_cols].fillna(0)
    output_cols = ['race_id','race_course_track'] + condition_ratio_cols
    return res_df[output_cols]

# Calculate track condition ratios
trackcond_df = calculate_track_condition_ratios(race_df)

# Display first few rows of the dataframe with new ratio columns
print("\nFirst few rows with track condition ratios:")
print(trackcond_df.shape)







First few rows with track condition ratios:
(2367, 11)


In [37]:
trackcond_df.sample(5)

,race_id,race_course_track,GOOD TO FIRM_ratio,WET FAST_ratio,GOOD_ratio,GOOD TO YIELDING_ratio,WET SLOW_ratio,FAST_ratio,YIELDING_ratio,YIELDING TO SOFT_ratio,SOFT_ratio
1107,2015-330,"Sha Tin_TURF - ""C+3"" COURSE",1,0,0,0,0,0,0,0,0
2028,2016-467,"Sha Tin_TURF - ""C"" COURSE",0,0,0,0,0,0,0,0,0
828,2015-052,"Happy Valley_TURF - ""C"" COURSE",0,0,1,0,0,0,0,0,0
568,2014-568,"Sha Tin_TURF - ""A"" COURSE",1,0,0,0,0,0,0,0,0
1487,2015-709,"Sha Tin_TURF - ""C"" COURSE",0,0,0,0,0,0,0,0,0


In [38]:
trackcond_df.to_parquet('../feature_data/trace_condition.parquet')

# horse features

In [39]:
# horse feature -  ratio between actual weight, declared weight
# horse feature - length behind winner statistics
# horse feature - running position statistics
# horse feature - jockey statistics - avg finishing position (<99)
# horse feature - trainer statistic - avg finishing position (<99)
# horse feature - failed to finish ration - num <99 / num_races


### horse time feature
### is horse getting faster, getting better position, running form in past 3, 5, 10 race
### is horse getting heavier 



### climate feature 
### 

In [40]:
horse_feats = horse_df[['horse_id']].copy()
horse_feats['awght_dwght_ratio'] = horse_df['clean_actual_weight'] / horse_df['clean_declared_horse_weight']
horse_feats['awght_dwght_delta'] = horse_df['clean_actual_weight'] - horse_df['clean_declared_horse_weight']

In [41]:
 # def get_groupbystat(data, groupby_feats, numfeats, stat):
 #    roundmetric_agg_rcp = [
 #        [groupby_feats, numfeats1, stat],
 #    ]

 #    res = project_utils.groupby_agg_execution(roundmetric_agg_rcp, data)['model']
 #    rename_dict = {}
 #    for c in res.columns.tolist():
 #        if c != groupby_feat:
 #            rename_dict[c] = c[6:] # remove 'model_' in column name
 #    res.rename(columns = rename_dict, inplace=True)
 #    return res

In [42]:
reload(project_utils)

<module 'project_tools.project_utils' from '/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py'>

# horse level groupby features

In [43]:
# horse level feature - can be directly merged with horse feature df 
groupby_feats1 = ['horse_id']
numfeats = ['clean_length_behind_winner', 'clean_actual_weight', 'clean_declared_horse_weight', 'clean_win_odds', 
            'running_position_1', 'running_position_2', 'running_position_3', 'running_position_4', 
            'running_position_5', 'running_position_6', 'race_distance']
stat = ['median', 'mean', 'std'] 

agg_recipies = [
    [groupby_feats1, numfeats, stat],
]
res_dfs = project_utils.groupby_agg_execution(agg_recipies, horse_race_df, stat)


generating statistic horse_id_clean_length_behind_winner_median
generating statistic horse_id_clean_length_behind_winner_mean
generating statistic horse_id_clean_length_behind_winner_std
generating statistic horse_id_clean_actual_weight_median
generating statistic horse_id_clean_actual_weight_mean
generating statistic horse_id_clean_actual_weight_std
generating statistic horse_id_clean_declared_horse_weight_median
generating statistic horse_id_clean_declared_horse_weight_mean
generating statistic horse_id_clean_declared_horse_weight_std
generating statistic horse_id_clean_win_odds_median
generating statistic horse_id_clean_win_odds_mean
generating statistic horse_id_clean_win_odds_std
generating statistic horse_id_running_position_1_median
generating statistic horse_id_running_position_1_mean
generating statistic horse_id_running_position_1_std
generating statistic horse_id_running_position_2_median
generating statistic horse_id_running_position_2_mean
generating statistic horse_id_run

In [44]:
horse_feats = res_dfs['horse_id']
horse_feats.shape

(2155, 34)

In [45]:
groupby_feats = ['horse_id','race_class']
numfeats = ['horse_name']
stat = ['count'] 

agg_recipies = [
    [groupby_feats, numfeats, stat],
]
res_dfs = project_utils.groupby_agg_execution(agg_recipies, horse_race_df, stat)
horse_class_feats = res_dfs['horse_id_race_class']

generating statistic horse_id_race_class_horse_name_count


In [46]:
horse_class_feats.sample(5)

,horse_id,race_class,horse_id_race_class_horse_name_count
2078,S198,Class 3,2
3704,V039,Class 5,5
263,A204,Class 4,7
298,A245,Class 3,1
383,K037,Class 4,9


In [47]:
horse_feats.to_parquet('../feature_data/horse_feats.parquet')
horse_class_feats.to_parquet('../feature_data/horse_class_feats.parquet')

# horse - race - track level groupby features

In [48]:
# groupby_feats1 = ['horse_id']
groupby_feats = ['horse_id', 'race_course_track']
numfeats = ['running_position_1', 'running_position_2', 'running_position_3', 'running_position_4', 
            'running_position_5', 'running_position_6', 'clean_finish_time', 'clean_win_odds']
stat = ['median', 'mean', 'std']  # {'ptp':np.ptp}]#{'sharp':project_utils.get_array_sharpe}]

agg_recipies = [
    [groupby_feats, numfeats, stat],
]
res_dfs = project_utils.groupby_agg_execution(agg_recipies, horse_race_df, stat)


generating statistic horse_id_race_course_track_running_position_1_median
generating statistic horse_id_race_course_track_running_position_1_mean
generating statistic horse_id_race_course_track_running_position_1_std
generating statistic horse_id_race_course_track_running_position_2_median
generating statistic horse_id_race_course_track_running_position_2_mean
generating statistic horse_id_race_course_track_running_position_2_std
generating statistic horse_id_race_course_track_running_position_3_median
generating statistic horse_id_race_course_track_running_position_3_mean
generating statistic horse_id_race_course_track_running_position_3_std
generating statistic horse_id_race_course_track_running_position_4_median
generating statistic horse_id_race_course_track_running_position_4_mean
generating statistic horse_id_race_course_track_running_position_4_std
generating statistic horse_id_race_course_track_running_position_5_median
generating statistic horse_id_race_course_track_running_po

In [49]:
res_dfs.keys()

dict_keys(['horse_id_race_course_track'])

In [50]:
# horse_rp_stat = res_dfs['horse_id']
horse_track_rp_stat = res_dfs['horse_id_race_course_track']

# print(horse_rp_stat.shape)
print(horse_track_rp_stat.shape)

(12704, 26)


In [51]:
# horse_rp_stat.to_parquet('../feature_data/horse_running_position_features.parquet')
horse_track_rp_stat.to_parquet('../feature_data/horse_track_running_position_features.parquet')

# time split feature gen

In [52]:
print(horse_race_df.columns.tolist())

['finishing_position', 'horse_number', 'horse_name', 'horse_id', 'jockey', 'trainer', 'actual_weight', 'declared_horse_weight', 'draw', 'length_behind_winner', 'running_position_1', 'running_position_2', 'running_position_3', 'running_position_4', 'finish_time', 'win_odds', 'running_position_5', 'running_position_6', 'race_id', 'clean_actual_weight', 'clean_declared_horse_weight', 'clean_length_behind_winner', 'clean_finish_time', 'clean_win_odds', 'clean_position', 'is_winner', 'is_top3', 'src', 'race_date', 'race_course', 'race_number', 'race_class', 'race_distance', 'track_condition', 'race_name', 'track', 'sectional_time', 'incident_report', 'race_course_track', 'clean_race_date']


In [53]:

def calculate_running_position_stats(df, horse_id_feature, feature_col='clean_position', specific_value = None):
    """
    Calculate running averages of a feature for each horse based on their last 3, 5 and 7 races
    
    Args:
        df: DataFrame containing horse race data with horse_id and feature columns
        feature_col: Name of the column to calculate running averages for (default: 'position')
        
    Returns:
        DataFrame with added columns for running averages
    """
    # Create copy to avoid modifying original
    result_df = df[[horse_id_feature]].copy()
    
    # Initialize new columns
    result_df[f'{feature_col}_mavg_3'] = np.nan
    result_df[f'{feature_col}_mavg_5'] = np.nan 
    result_df[f'{feature_col}_mavg_7'] = np.nan

    # Get unique horses
    horses = df[horse_id_feature].unique()
    
    # Calculate running averages for each horse
    for horse in horses:
        # Get all races for this horse in chronological order
        horse_mask = df['horse_id'] == horse
        
        # Get feature values, replacing 99 with nan
        if specific_value is not None:
            values = df.loc[horse_mask, feature_col].replace(99, np.nan)
        else:
            values = df.loc[horse_mask, feature_col]
            
        # Calculate running means with different windows
        mavg_3 = values.rolling(window=3, min_periods=3).mean()
        mavg_5 = values.rolling(window=5, min_periods=5).mean()
        mavg_7 = values.rolling(window=7, min_periods=7).mean()
        
        # Store results
        result_df.loc[horse_mask, f'{feature_col}_mavg_3'] = mavg_3
        result_df.loc[horse_mask, f'{feature_col}_mavg_5'] = mavg_5
        result_df.loc[horse_mask, f'{feature_col}_mavg_7'] = mavg_7
        
    return result_df


In [54]:
horse_race_df = horse_race_df.sort_values(by=['clean_race_date','race_id'], ascending=True).reset_index(drop=True)


In [55]:
horse_mavgpos_df= calculate_running_position_stats(horse_race_df, 'horse_id', 
                                                       feature_col='clean_position', specific_value = 99)

In [56]:
horse_mavgpos_df.sample(5)

,horse_id,clean_position_mavg_3,clean_position_mavg_5,clean_position_mavg_7
9057,S440,11,NaN,NaN
21249,V069,6,4,4
23023,V088,5,6,7
9065,N140,8,8,7
2540,P008,9,NaN,NaN


In [57]:
print(horse_mavgpos_df.shape, horse_race_df.shape)

(29520, 4) (29520, 40)


In [58]:
mavg_cols = [c for c in horse_mavgpos_df.columns if 'mavg' in c]
print(horse_race_df.shape)
horse_race_df = pd.concat([horse_race_df, horse_mavgpos_df[mavg_cols]], axis=1)
print(horse_race_df.shape)


(29520, 40)
(29520, 43)


In [59]:
groupby_feats = ['horse_id']
use_cols = ['horse_id', 'race_id', 'clean_race_date', 'clean_position']
group_objects = horse_race_df.groupby(groupby_feats)
for g in group_objects:
    break

In [60]:
g[1][use_cols]

,horse_id,race_id,clean_race_date,clean_position
22649,A001,2016-250,20161211,11
23445,A001,2016-313,20170104,11
24279,A001,2016-380,20170130,6


In [61]:
list(group_objects)[0][1][use_cols]

,horse_id,race_id,clean_race_date,clean_position
22649,A001,2016-250,20161211,11
23445,A001,2016-313,20170104,11
24279,A001,2016-380,20170130,6


# ordinal encoding

In [76]:
reload(project_utils)

<module 'project_tools.project_utils' from '/home/yifan/playground/hk_horse_racing/project_tools/project_utils.py'>

In [71]:
horse_race_df.shape

(29520, 44)

In [77]:
basic_cat_features = ['jockey', 'trainer'] +  ['race_course', 'race_course_track', 
                                               'race_class',  'track_condition']

basic_cat_ordinal_df = project_utils.ordinal_encoder(horse_race_df, basic_cat_features)

In [135]:
basic_cat_ordinal_df.sample(5)

,jockey,trainer,race_course,race_course_track,race_class,track_condition
5215,3,8,1,8,4,2
4767,19,3,1,7,2,0
22149,6,18,1,9,3,2
10130,21,6,1,7,2,2
9964,21,6,0,4,0,0


In [79]:
basic_cat_ordinal_df.to_parquet('../feature_data/basic_cat_ordinal_features.parquet')

In [136]:
horse_race_df.to_parquet('../feature_data/horse_race_df.parquet')

# train val seperation

In [72]:
horse_race_df[basic_cat_features].sample(5)

,jockey,trainer,race_course,race_course_track,race_class,track_condition
6925,J Moreira,W Y So,Happy Valley,"Happy Valley_TURF - ""C"" COURSE",Class 4,GOOD TO FIRM
14471,K C Leung,K L Man,Sha Tin,"Sha Tin_TURF - ""B+2"" COURSE",Class 4,GOOD
10925,K K Chiong,C H Yip,Sha Tin,"Sha Tin_TURF - ""C"" COURSE",Class 3,GOOD
21889,S Clipperton,P F Yiu,Happy Valley,"Happy Valley_TURF - ""C"" COURSE",Class 4,GOOD
22868,S Clipperton,T K Ng,Sha Tin,Sha Tin_ALL WEATHER TRACK,Class 5,GOOD


In [62]:
horse_race_df['year'] = horse_race_df['race_date'].apply(lambda x:x[0:4])
horse_race_df['year'].sample(5)

22073    2016
2354     2014
3364     2015
15765    2016
12031    2015
Name: year, dtype: object

In [63]:
horse_race_df['year'].value_counts()

year
2015    10006
2016     9869
2017     6288
2014     3357
Name: count, dtype: int64

(29520, 44)

In [64]:
train_years = ['2014','2015','2016']
val_years = ['2017']
train_horse_race_df = horse_race_df[horse_race_df['year'].isin(train_years)].reset_index(drop=True)
val_horse_race_df = horse_race_df[horse_race_df['year'].isin(val_years)].reset_index(drop=True)

In [65]:
train_horse_race_df.to_parquet('../feature_data/train_horse_race_df.parquet')
val_horse_race_df.to_parquet('../feature_data/val_horse_race_df.parquet')


# horse, jockey, trainer performance features in train dataset

In [66]:
groupby_feats1 = ['horse_id']
groupby_feats2 = ['jockey'] 
groupby_feats3 = ['trainer']
numfeats = ['clean_position']
stat = ['median', 'mean', 'std', 'count']  # {'ptp':np.ptp}]#{'sharp':project_utils.get_array_sharpe}]


agg_recipies = [
    [groupby_feats1, numfeats, stat],
    [groupby_feats2, numfeats, stat],
    [groupby_feats3, numfeats, stat],
]
res_dfs = project_utils.groupby_agg_execution(agg_recipies, train_horse_race_df, stat)

generating statistic horse_id_clean_position_median
generating statistic horse_id_clean_position_mean
generating statistic horse_id_clean_position_std
generating statistic horse_id_clean_position_count
generating statistic jockey_clean_position_median
generating statistic jockey_clean_position_mean
generating statistic jockey_clean_position_std
generating statistic jockey_clean_position_count
generating statistic trainer_clean_position_median
generating statistic trainer_clean_position_mean
generating statistic trainer_clean_position_std
generating statistic trainer_clean_position_count


In [90]:
res_dfs.keys()

dict_keys(['horse_id', 'jockey', 'trainer'])

In [91]:
train_horse_positions_df = res_dfs['horse_id']
train_jockey_positions_df = res_dfs['jockey']
train_trainer_positions_df = res_dfs['trainer']

print(train_horse_positions_df.shape)
print(train_jockey_positions_df.shape)
print(train_trainer_positions_df.shape)

(1895, 5)
(99, 5)
(87, 5)


In [92]:
train_horse_positions_df.to_parquet('../feature_data/train_horse_positions_df.parquet')
train_jockey_positions_df.to_parquet('../feature_data/train_jockey_positions_df.parquet')
train_trainer_positions_df.to_parquet('../feature_data/train_trainer_positions_df.parquet')


In [138]:
train_hids = train_horse_race_df['horse_id'].unique().tolist()
val_hids = val_horse_race_df['horse_id'].unique().tolist()

interset_hids = project_utils.list_intersection(val_hids, train_hids)
interset_ratio = len(interset_hids)/len(train_hids)
print(f"{interset_ratio:.3}")

0.476


In [142]:
# jockey overlapping ratio
train_jids = train_horse_race_df['jockey'].unique().tolist()
val_jids = val_horse_race_df['jockey'].unique().tolist()

interset_jids = project_utils.list_intersection(val_jids, train_jids)
interset_ratio = len(interset_jids)/len(train_jids)
print(f"{interset_ratio:.3}")

0.374


In [80]:
project_utils.analyze_dataframe(horse_rp_stat)

,feature,missing_count,missing_ratio,mean,median,min,25%,75%,max,std
0,horse_id,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,horse_id_running_position_1_median,7,0.003238,6.915313,7.000000,1.0,5.000000,9.000000,14.000000,2.955197
2,horse_id_running_position_1_mean,7,0.003238,7.031071,7.142857,1.0,5.167183,8.924286,14.000000,2.580133
3,horse_id_running_position_1_count,0,0.000000,13.679001,12.000000,0.0,5.000000,21.000000,50.000000,10.105534
4,horse_id_running_position_1_std,170,0.078631,2.845479,2.900767,0.0,2.308828,3.419893,7.071068,0.931072
5,horse_id_running_position_2_median,7,0.003238,6.945012,7.000000,1.0,5.000000,9.000000,14.000000,2.953700
6,horse_id_running_position_2_mean,7,0.003238,7.065828,7.066667,1.0,5.246622,8.888889,14.000000,2.553170
7,horse_id_running_position_2_count,0,0.000000,13.672525,12.000000,0.0,5.000000,21.000000,50.000000,10.103402
8,horse_id_running_position_2_std,170,0.078631,2.899432,2.980085,0.0,2.357671,3.493714,7.778175,0.935241
9,horse_id_running_position_3_median,7,0.003238,7.151740,7.000000,1.0,5.000000,9.000000,14.000000,2.871140


In [76]:
horse_track_rp_stat.columns

Index(['horse_id', 'race_course_track',
       'horse_id_race_course_track_running_position_1_median',
       'horse_id_race_course_track_running_position_1_mean',
       'horse_id_race_course_track_running_position_1_count',
       'horse_id_race_course_track_running_position_1_std',
       'horse_id_race_course_track_running_position_2_median',
       'horse_id_race_course_track_running_position_2_mean',
       'horse_id_race_course_track_running_position_2_count',
       'horse_id_race_course_track_running_position_2_std',
       'horse_id_race_course_track_running_position_3_median',
       'horse_id_race_course_track_running_position_3_mean',
       'horse_id_race_course_track_running_position_3_count',
       'horse_id_race_course_track_running_position_3_std',
       'horse_id_race_course_track_running_position_4_median',
       'horse_id_race_course_track_running_position_4_mean',
       'horse_id_race_course_track_running_position_4_count',
       'horse_id_race_course_track_r

In [77]:
horse_track_rp_stat.sample(5)

,horse_id,race_course_track,horse_id_race_course_track_running_position_1_median,horse_id_race_course_track_running_position_1_mean,horse_id_race_course_track_running_position_1_count,horse_id_race_course_track_running_position_1_std,horse_id_race_course_track_running_position_2_median,horse_id_race_course_track_running_position_2_mean,horse_id_race_course_track_running_position_2_count,horse_id_race_course_track_running_position_2_std,horse_id_race_course_track_running_position_3_median,horse_id_race_course_track_running_position_3_mean,horse_id_race_course_track_running_position_3_count,horse_id_race_course_track_running_position_3_std,horse_id_race_course_track_running_position_4_median,horse_id_race_course_track_running_position_4_mean,horse_id_race_course_track_running_position_4_count,horse_id_race_course_track_running_position_4_std
1303,M086,Sha Tin_ALL WEATHER TRACK,2.0,2.00,2,1.414214,3.0,3.0,2,0.000000,5.5,5.5,2,3.535534,9.0,9.0,2,7.071068
6803,S317,"Happy Valley_TURF - ""B"" COURSE",6.5,6.50,2,2.121320,7.0,7.0,2,2.828427,6.5,6.5,2,0.707107,NaN,NaN,0,NaN
11805,V209,"Happy Valley_TURF - ""B"" COURSE",7.5,6.75,4,2.629956,7.0,6.5,4,1.732051,10.0,9.5,4,3.000000,11.0,11.0,1,NaN
11025,V072,"Happy Valley_TURF - ""C+3"" COURSE",10.0,10.00,1,NaN,11.0,11.0,1,NaN,11.0,11.0,1,NaN,6.0,6.0,1,NaN
9736,T313,"Happy Valley_TURF - ""C+3"" COURSE",8.0,8.00,1,NaN,8.0,8.0,1,NaN,12.0,12.0,1,NaN,NaN,NaN,0,NaN


In [78]:
horse_rp_stat.sample(5)

,horse_id,horse_id_running_position_1_median,horse_id_running_position_1_mean,horse_id_running_position_1_count,horse_id_running_position_1_std,horse_id_running_position_2_median,horse_id_running_position_2_mean,horse_id_running_position_2_count,horse_id_running_position_2_std,horse_id_running_position_3_median,horse_id_running_position_3_mean,horse_id_running_position_3_count,horse_id_running_position_3_std,horse_id_running_position_4_median,horse_id_running_position_4_mean,horse_id_running_position_4_count,horse_id_running_position_4_std
1127,S233,4.0,4.400000,5,2.966479,6.0,7.000000,5,4.527693,10.0,10.000000,5,2.121320,NaN,NaN,0,NaN
1474,T159,11.0,10.281250,32,2.617982,11.0,10.187500,32,2.361485,9.0,8.875000,32,2.756225,9.0,8.285714,21,3.874827
1618,T311,8.0,8.238095,21,3.096849,9.0,8.190476,21,3.026864,7.0,7.047619,21,2.597618,4.0,4.333333,21,2.708013
1642,T337,9.0,9.142857,7,2.672612,8.0,8.714286,7,2.563480,7.0,8.285714,7,3.039424,10.0,9.571429,7,1.618347
2088,V381,6.0,7.727273,11,3.523944,7.0,6.818182,11,3.736795,8.0,6.454545,11,3.615623,8.0,8.000000,10,3.231787


In [65]:
hid = 'T288' #'T403' #'P347'
cid = 'Sha Tin_TURF - "A" COURSE' #horse_track_stat['race_course_track'].values[10356]
print(hid, cid)
check_df = horse_race_df[(horse_race_df['horse_id']==hid) & (horse_race_df['race_course_track']==cid)]
# check_df = horse_race_df[(horse_race_df['horse_id']==hid)]

check_df.shape

T288 Sha Tin_TURF - "A" COURSE


(2, 37)

In [66]:
use_cols = ['horse_id', 'race_course_track', 'running_position_1', 'running_position_2', 
            'running_position_3', 'running_position_4']
check_df[use_cols]

,horse_id,race_course_track,running_position_1,running_position_2,running_position_3,running_position_4
17404,T288,"Sha Tin_TURF - ""A"" COURSE",5.0,6.0,7.0,10.0
20638,T288,"Sha Tin_TURF - ""A"" COURSE",5.0,4.0,5.0,8.0


In [39]:
hid = 'P347'
cid = 'Happy Valley_TURF - "A" COURSE'  #horse_track_stat['race_course_track'].values[4390]
print(hid, cid)
# check_df = horse_race_df[(horse_race_df['horse_id']==hid) & (horse_race_df['race_course_track']==hid)]
check_df2 = horse_race_df[(horse_race_df['horse_id']==hid)]

check_df2.shape

P347 Happy Valley_TURF - "A" COURSE


(9, 37)

In [40]:
use_cols = ['horse_id', 'race_course_track', 'running_position_1', 'running_position_2', 
            'running_position_3', 'running_position_4']
check_df2[use_cols]

,horse_id,race_course_track,running_position_1,running_position_2,running_position_3,running_position_4
154,P347,Sha Tin_ALL WEATHER TRACK,4.0,4.0,10.0,NaN
2708,P347,Sha Tin_ALL WEATHER TRACK,1.0,3.0,10.0,NaN
3800,P347,Sha Tin_ALL WEATHER TRACK,1.0,2.0,3.0,NaN
4719,P347,Sha Tin_ALL WEATHER TRACK,3.0,2.0,9.0,NaN
5459,P347,"Happy Valley_TURF - ""A"" COURSE",3.0,4.0,1.0,NaN
7050,P347,"Happy Valley_TURF - ""C"" COURSE",7.0,4.0,6.0,NaN
7799,P347,"Happy Valley_TURF - ""B"" COURSE",1.0,1.0,7.0,NaN
8953,P347,"Happy Valley_TURF - ""C"" COURSE",6.0,5.0,11.0,NaN
9703,P347,"Happy Valley_TURF - ""A"" COURSE",2.0,6.0,11.0,NaN


In [81]:
check_df =  horse_rp_stat[pd.isnull(horse_rp_stat['horse_id_running_position_1_std'])]
check_df.head(5)

,horse_id,horse_id_running_position_1_median,horse_id_running_position_1_mean,horse_id_running_position_1_count,horse_id_running_position_1_std,horse_id_running_position_2_median,horse_id_running_position_2_mean,horse_id_running_position_2_count,horse_id_running_position_2_std,horse_id_running_position_3_median,horse_id_running_position_3_mean,horse_id_running_position_3_count,horse_id_running_position_3_std,horse_id_running_position_4_median,horse_id_running_position_4_mean,horse_id_running_position_4_count,horse_id_running_position_4_std
12,A013,4.0,4.0,1,NaN,4.0,4.0,1,NaN,6.0,6.0,1,NaN,NaN,NaN,0,NaN
17,A019,10.0,10.0,1,NaN,11.0,11.0,1,NaN,10.0,10.0,1,NaN,NaN,NaN,0,NaN
29,A033,4.0,4.0,1,NaN,5.0,5.0,1,NaN,9.0,9.0,1,NaN,NaN,NaN,0,NaN
39,A045,13.0,13.0,1,NaN,13.0,13.0,1,NaN,6.0,6.0,1,NaN,NaN,NaN,0,NaN
48,A054,10.0,10.0,1,NaN,10.0,10.0,1,NaN,13.0,13.0,1,NaN,14.0,14.0,1,NaN


In [ ]:
groupby_feats1 = ['horse_id']
groupby_feats2 = ['horse_id', 'race_course_track']
numfeats = ['running_position_1', 'running_position_2', 'running_position_3', 'running_position_4']
stat = ['sum', 'mean', 'count', 'std',
         {'consistency': project_utils.get_array_sharpe}] 

In [259]:
horse_df['declared_horse_weight'].values[0]

'1032'

# initial analysis

In [289]:
print(horse_df.shape)
print(race_df.shape)


(30189, 25)
(2367, 13)


In [290]:
print(horse_df.columns)
print(race_df.columns)


Index(['finishing_position', 'horse_number', 'horse_name', 'horse_id',
       'jockey', 'trainer', 'actual_weight', 'declared_horse_weight', 'draw',
       'length_behind_winner', 'running_position_1', 'running_position_2',
       'running_position_3', 'running_position_4', 'finish_time', 'win_odds',
       'running_position_5', 'running_position_6', 'race_id', 'clean_position',
       'clean_actual_weight', 'clean_declared_horse_weight',
       'clean_length_behind_winner', 'isin_winner', 'isin_top3'],
      dtype='object')
Index(['src', 'race_date', 'race_course', 'race_number', 'race_id',
       'race_class', 'race_distance', 'track_condition', 'race_name', 'track',
       'sectional_time', 'incident_report', 'race_course_track'],
      dtype='object')


In [86]:
horse_df.sample(5)

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id,clean_position,clean_actual_weight,clean_declared_horse_weight,clean_length_behind_winner,isin_winner,isin_top3
7000,12,9.0,GLORIOUS RYDER,P279,N Callan,R Gibson,126,1164,14,32,12.0,12.0,13.0,12.0,1.43.92,9.6,NaN,NaN,2014-548,12,126.0,1164.0,32.00,0,0
13660,8,10.0,TRIBAL GLORY,S395,G Mosse,P F Yiu,125,1131,8,3,11.0,9.0,8.0,8.0,1.23.78,99,NaN,NaN,2015-291,8,125.0,1131.0,3.00,0,0
9012,1,7.0,RAINBOW CHIC,P118,Z Purton,C Fownes,120,1125,5,-,9.0,8.0,8.0,8.0,1.48.30,8.2,1.0,NaN,2014-706,1,120.0,1125.0,0.00,1,1
1092,10,8.0,FOREVER FUN,S130,H W Lai,K W Lui,120,1080,6,7-3/4,2.0,3.0,10.0,NaN,1.10.50,30,NaN,NaN,2014-089,10,120.0,1080.0,7.75,0,0
9362,5,2.0,RACING HERO,N157,Z Purton,P O'Sullivan,132,1185,4,3,4.0,5.0,6.0,5.0,1.22.05,8,NaN,NaN,2014-732,5,132.0,1185.0,3.00,0,0


In [48]:
exclude_cols =['incident_report']
race_df.drop(exclude_cols, axis=1).sample(5)

,src,race_date,race_course,race_number,race_id,race_class,race_distance,track_condition,race_name,track,sectional_time,race_course_track
667,20150603-6.html,2015-06-03,Happy Valley,6,2014-668,Class 3,1000,GOOD TO FIRM,THE CRICKET CLUB VALLEY STAKES (HANDICAP),"TURF - ""A"" COURSE",12.64 21.39 23.02,"Happy Valley_TURF - ""A"" COURSE"
470,20150318-3.html,2015-03-18,Happy Valley,3,2014-471,Class 4,1200,GOOD,FORTRESS HILL HANDICAP,"TURF - ""C"" COURSE",23.77 23.22 23.06,"Happy Valley_TURF - ""C"" COURSE"
1761,20161120-8.html,2016-11-20,Sha Tin,8,2016-201,Group Two,1200,GOOD,THE BOCHK WEALTH MANAGEMENT JOCKEY CLUB SPRINT,"TURF - ""B+2"" COURSE",23.46 21.97 22.83,"Sha Tin_TURF - ""B+2"" COURSE"
1454,20160601-1.html,2016-06-01,Happy Valley,1,2015-678,Class 5,1650,GOOD,MOUNT CAMERON HANDICAP,"TURF - ""C+3"" COURSE",27.75 23.40 25.60 24.09,"Happy Valley_TURF - ""C+3"" COURSE"
805,20150916-1.html,2015-09-16,Happy Valley,1,2015-029,Class 5,1200,GOOD,CHEUNG HONG HANDICAP,"TURF - ""B"" COURSE",23.83 22.89 24.29,"Happy Valley_TURF - ""B"" COURSE"


In [76]:
reload(project_utils)
project_utils.analyze_dataframe(horse_df)

/home/yifan/anaconda3/envs/py310/lib/python3.10/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,feature,missing_count,missing_ratio,value_types,mean,median,min,25%,75%,max,std
0,finishing_position,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,horse_number,0,0.000000,[float],6.883367,7.00,1.00,4.000000,10.00,14.0,3.752504
2,horse_name,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,horse_id,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jockey,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,trainer,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,actual_weight,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,declared_horse_weight,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,draw,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,length_behind_winner,0,0.000000,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
project_utils.analyze_dataframe(race_df)

,feature,missing_count,missing_ratio,value_types,mean,median,min,25%,75%,max,std
0,src,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,race_date,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,race_course,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,race_number,0,0,[int],5,5,1,3,7,11,3
4,race_id,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,race_class,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,race_distance,0,0,[int],1415,1400,1000,1200,1650,2400,284
7,track_condition,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,race_name,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,track,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
race_df['clean_race_date'].sample(5).values[0:5].tolist()

[20170412, 20170222, 20150701, 20160522, 20151101]

In [50]:
horse_df['finish_time'].sample(5)

2584     1.41.91
28093    1.40.12
30123    1.51.25
24984    0.58.41
16637    1.51.13
Name: finish_time, dtype: object

In [51]:
horse_df['finish_time'].value_counts()

finish_time
---        78
1.10.72    59
1.10.37    58
1.10.45    56
1.10.42    56
           ..
1.43.73     1
1.44.51     1
1.44.18     1
1.00.74     1
1.25.26     1
Name: count, Length: 4175, dtype: int64

In [53]:
horse_df[horse_df['finish_time']=='---'].shape

(78, 26)

In [33]:
horse_df['length_behind_winner'].sample(5)

25304    2-1/4
15497        -
24597    6-1/2
1829     6-1/2
4088     3-1/2
Name: length_behind_winner, dtype: object

In [39]:
horse_df['length_behind_winner'].unique().tolist()

['-',
 '2',
 '4-1/4',
 '5-1/2',
 '5-3/4',
 '6-1/4',
 '6-3/4',
 '7-3/4',
 '10-1/4',
 '11',
 '---',
 '2-1/4',
 '2-1/2',
 '2-3/4',
 '3-1/2',
 '4',
 '4-1/2',
 '4-3/4',
 '5',
 '7-1/4',
 '9',
 '10-3/4',
 '1-1/4',
 '1-3/4',
 '7',
 '8-1/2',
 '12-1/2',
 'N',
 '1',
 '3-3/4',
 '6',
 '3',
 '3-1/4',
 '5-1/4',
 '16-1/4',
 '20',
 'SH',
 'HD',
 '1/2',
 '7-1/2',
 '8-1/4',
 '12-1/4',
 '17-1/2',
 '1-1/2',
 '11-1/4',
 '12-3/4',
 '8',
 '6-1/2',
 '9-1/4',
 '9-1/2',
 '10-1/2',
 '21-1/4',
 '36-1/4',
 '13-3/4',
 '10',
 '3/4',
 '11-1/2',
 '24',
 '25-1/4',
 '12',
 '16',
 '26',
 '8-3/4',
 '16-1/2',
 '11-3/4',
 '13-1/4',
 '14-3/4',
 '15',
 '29-1/4',
 '19-1/2',
 '9-3/4',
 '19-1/4',
 '22-1/4',
 '22-1/2',
 '13-1/2',
 '16-3/4',
 '17',
 '14',
 '13',
 '22',
 '15-1/4',
 '39',
 '20-3/4',
 '22-3/4',
 '19-3/4',
 '21-3/4',
 '55-1/2',
 '14-1/2',
 '14-1/4',
 '15-1/2',
 '26-3/4',
 '20-1/4',
 '30',
 '38-1/2',
 'NOSE',
 '46-1/4',
 '18-3/4',
 '33-3/4',
 '24-3/4',
 '15-3/4',
 '23',
 '19',
 '21-1/2',
 '23-3/4',
 '29-3/4',
 '27-3/4',

In [208]:
lbw_vals = horse_df['length_behind_winner'].unique().tolist()
non_numeric_lbw = ['---', '-', 'N', 'SH', 'HD', 'NOSE', '+NOSE', 'ML', '+SH', 'TO', '+1/2']
numeric_lbw = [v for v in lbw_vals if v not in non_numeric_lbw]
print(numeric_lbw)

['2', '4-1/4', '5-1/2', '5-3/4', '6-1/4', '6-3/4', '7-3/4', '10-1/4', '11', '2-1/4', '2-1/2', '2-3/4', '3-1/2', '4', '4-1/2', '4-3/4', '5', '7-1/4', '9', '10-3/4', '1-1/4', '1-3/4', '7', '8-1/2', '12-1/2', '1', '3-3/4', '6', '3', '3-1/4', '5-1/4', '16-1/4', '20', '1/2', '7-1/2', '8-1/4', '12-1/4', '17-1/2', '1-1/2', '11-1/4', '12-3/4', '8', '6-1/2', '9-1/4', '9-1/2', '10-1/2', '21-1/4', '36-1/4', '13-3/4', '10', '3/4', '11-1/2', '24', '25-1/4', '12', '16', '26', '8-3/4', '16-1/2', '11-3/4', '13-1/4', '14-3/4', '15', '29-1/4', '19-1/2', '9-3/4', '19-1/4', '22-1/4', '22-1/2', '13-1/2', '16-3/4', '17', '14', '13', '22', '15-1/4', '39', '20-3/4', '22-3/4', '19-3/4', '21-3/4', '55-1/2', '14-1/2', '14-1/4', '15-1/2', '26-3/4', '20-1/4', '30', '38-1/2', '46-1/4', '18-3/4', '33-3/4', '24-3/4', '15-3/4', '23', '19', '21-1/2', '23-3/4', '29-3/4', '27-3/4', '21', '28-1/4', '25', '18-1/2', '32', '32-1/2', '27-1/2', '42', '26-1/4', '31-1/2', '40-1/2', '33-1/2', '17-1/4', '17-3/4', '18', '28-3/4', '

In [209]:
lbw_t2n_dict = {}
lbw_t2n_dict['-'] = 0
lbw_t2n_dict['N'] = 0.3
lbw_t2n_dict['SH'] = 0.1
lbw_t2n_dict['HD'] = 0.25
lbw_t2n_dict['NOSE'] = 0.05
lbw_t2n_dict['+NOSE'] = 0.75
lbw_t2n_dict['ML'] = 50
lbw_t2n_dict['+SH'] = 0.15
lbw_t2n_dict['TO'] = 0.1
lbw_t2n_dict['+1/2'] = 99
lbw_t2n_dict['---'] = 99

lbw_t2n_dict

{'-': 0,
 'N': 0.3,
 'SH': 0.1,
 'HD': 0.25,
 'NOSE': 0.05,
 '+NOSE': 0.75,
 'ML': 50,
 '+SH': 0.15,
 'TO': 0.1,
 '+1/2': 99,
 '---': 99}

In [207]:
# to check ---, -, 'N', 'SH', 'HD', 'NOSE', '+NOSE', 'ML', '+SH', 'TO', '+1/2'
# --- 99

lbw_t2n_dict = {}
lbw_t2n_dict['-'] = 0
lbw_t2n_dict['N'] = 0.3
lbw_t2n_dict['SH'] = 0.1
lbw_t2n_dict['HD'] = 0.25
lbw_t2n_dict['NOSE'] = 0.05
lbw_t2n_dict['+NOSE'] = 0.75
lbw_t2n_dict['ML'] = 50
lbw_t2n_dict['+SH'] = 0.15
lbw_t2n_dict['TO'] = 0.1
lbw_t2n_dict['+1/2'] = 99
lbw_t2n_dict['---'] = 99


    

use_cols = ['finishing_position', 'clean_position', 'length_behind_winner']
non_numeric_lbw = ['---', '-', 'N', 'SH', 'HD', 'NOSE', '+NOSE', 'ML', '+SH', 'TO', '+1/2']
# horse_df[horse_df['length_behind_winner'].isin(non_numeric_lbw)][use_cols].sample(5)
horse_df[horse_df['length_behind_winner']=='+1/2'][use_cols]

,finishing_position,clean_position,length_behind_winner
4456,DISQ,99,+1/2


In [42]:


use_cols = ['finishing_position', 'clean_position', 'length_behind_winner']
horse_df[horse_df['length_behind_winner']=='NOSE'][use_cols].sample(5)

,clean_position,length_behind_winner
22300,2,NOSE
24924,2,NOSE
18179,2,NOSE
9241,2,NOSE
25543,2,NOSE


In [100]:
horse_df[horse_df['clean_position']==4][use_cols].sample(5)

,clean_position,length_behind_winner
7906,4,2-3/4
18671,4,2
14241,4,2-1/4
14837,4,4-1/4
4806,4,1-3/4


In [36]:
horse_df['clean_position'].describe()

count    30189.000000
mean         8.862334
std         14.070619
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         99.000000
Name: clean_position, dtype: float64

In [87]:
pd.isnull(horse_df['finishing_position']).sum()

0

In [88]:
horse_df['finishing_position'].value_counts()

finishing_position
1        2361
2        2354
3        2350
6        2346
5        2341
4        2340
7        2339
8        2329
9        2305
10       2260
11       2186
12       2020
13        995
14        838
WV        461
WV-A      102
4 DH       35
PU         29
UR         26
3 DH       20
5 DH       18
WX         16
2 DH       14
6 DH       14
8 DH       12
1 DH       12
FE         10
WX-A       10
10 DH       8
9 DH        8
7 DH        8
DNF         7
TNP         6
11 DH       4
12 DH       2
NA          2
DISQ        1
Name: count, dtype: int64

In [89]:
horse_df[horse_df['finishing_position'].str.contains('DH')]['finishing_position']

297      9 DH
298      9 DH
592      2 DH
593      2 DH
752      1 DH
         ... 
29328    9 DH
29665    5 DH
29666    5 DH
29945    4 DH
29946    4 DH
Name: finishing_position, Length: 155, dtype: object

In [79]:
horse_df['running_position_3'].describe()

count    29542.000000
mean         6.826992
std          3.724886
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: running_position_3, dtype: float64

In [80]:
horse_df['running_position_4'].describe()

count    16618.000000
mean         6.942472
std          3.798080
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: running_position_4, dtype: float64

In [77]:
horse_df['jockey'].value_counts()

jockey
J Moreira    1995
K Teetan     1761
Z Purton     1651
D Whyte      1608
N Callan     1590
             ... 
K Manning       1
T Ono           1
M Nunes         1
J Spencer       1
T Jarnet        1
Name: count, Length: 106, dtype: int64

In [78]:
horse_df['trainer'].value_counts()

trainer
A S Cruz          1751
C Fownes          1724
C H Yip           1692
J Moore           1654
Y S Tsui          1652
J Size            1499
C S Shum          1466
W Y So            1438
P F Yiu           1410
A T Millard       1349
K L Man           1337
T P Yung          1263
D E Ferraris      1251
P O'Sullivan      1195
A Lee             1182
C W Chang         1136
K W Lui           1132
R Gibson          1103
D J Hall          1090
D Cruz            1012
L Ho              1012
T K Ng             715
S Woods            509
A Schutz           496
H Fujiwara           8
N Hori               7
Y Ikee               5
D A Hayes            4
A Fabre              4
A P O'Brien          4
J Lau                3
M Saito              3
Sir M R Stoute       3
Barande-Barbe        3
P Bary               2
G Allendorf          2
K C Chong            2
Y C Fung             2
de Royer Dupre       2
G Enebish            2
M C Tam              2
M Sakaguchi          2
T Ozeki              2
K C

In [37]:
race_df['incident_report'].sample(1).values[0]

'\n                TRAVEL BROTHER was withdrawn on 5.4.16 by order of the Stewards acting on veterinary advice (lame left fore).  BORN IN CHINA was also withdrawn on 5.4.16 by order of the Stewards acting on veterinary advice (lame right fore).  Before being allowed to race again, TRAVEL BROTHER and BORN IN CHINA will be subjected to an official veterinary examination.\nFrom the outside barriers, MR GENUINE and DISCIPLES TWELVE were shifted across behind runners shortly after the start.\nApproaching the 800 Metres, MR GENUINE was retired from the race.  A veterinary inspection of MR GENUINE immediately following the race found that horse to be lame in its right hind leg.  Before being allowed to race again, MR GENUINE will be subjected to an official veterinary examination.\nPABLOSKY and PACKING LLAREGYB were sent for sampling.\n'

,src,race_date,race_course,race_number,race_id,race_class,race_distance,track_condition,race_name,track,sectional_time,race_course_track,GOOD TO FIRM_ratio,WET FAST_ratio,GOOD_ratio,GOOD TO YIELDING_ratio,WET SLOW_ratio,FAST_ratio,YIELDING_ratio,YIELDING TO SOFT_ratio,SOFT_ratio
2256,20170604-1.html,2017-06-04,Sha Tin,1,2016-697,Griffin Race,1200,GOOD TO FIRM,TATE'S CAIRN PLATE,"TURF - ""B"" COURSE",25.54 23.15 22.28,"Sha Tin_TURF - ""B"" COURSE",0.142857,0.00000,0.857143,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2151,20170423-3.html,2017-04-23,Sha Tin,3,2016-591,Class 4,1650,WET SLOW,AGE-FRIENDLY CITY HANDICAP,ALL WEATHER TRACK,27.96 23.43 24.20 23.44,Sha Tin_ALL WEATHER TRACK,0.000000,0.04878,0.808362,0.000000,0.076655,0.066202,0.000000,0.0,0.0
731,20150627-6.html,2015-06-27,Sha Tin,6,2014-731,Class 3,1000,GOOD TO FIRM,LEI YUE MUN PUBLIC RIDING SCHOOL HANDICAP,"TURF - ""C+3"" COURSE",13.05 20.42 22.02,"Sha Tin_TURF - ""C+3"" COURSE",0.533865,0.00000,0.406375,0.051793,0.000000,0.000000,0.007968,0.0,0.0
1267,20160320-10.html,2016-03-20,Sha Tin,10,2015-499,Class 3,1600,GOOD,COLLECTION HANDICAP,"TURF - ""A"" COURSE",25.10 23.09 24.18 23.09,"Sha Tin_TURF - ""A"" COURSE",0.550781,0.00000,0.382812,0.050781,0.000000,0.000000,0.015625,0.0,0.0
1228,20160302-8.html,2016-03-02,Happy Valley,8,2015-452,Class 3,1650,GOOD,SIU SAI WAN HANDICAP,"TURF - ""C+3"" COURSE",28.00 23.31 25.07 23.81,"Happy Valley_TURF - ""C+3"" COURSE",0.342105,0.00000,0.638158,0.019737,0.000000,0.000000,0.000000,0.0,0.0


In [51]:
race_df['track_condition'].value_counts()

track_condition
GOOD                1296
GOOD TO FIRM         860
GOOD TO YIELDING     126
YIELDING              28
WET SLOW              22
FAST                  19
WET FAST              14
YIELDING TO SOFT       1
SOFT                   1
Name: count, dtype: int64

In [53]:
race_df['race_course'].value_counts()


race_course
Sha Tin         1513
Happy Valley     854
Name: count, dtype: int64

In [74]:
race_df['race_class'].value_counts()

race_class
Class 4                        859
Class 3                        756
Class 5                        339
Class 2                        223
Class 1                         51
Group One                       32
Hong Kong Group Three           25
Griffin Race                    16
Group Two                       14
Group Three                     12
Class 4 (Restricted)             9
Hong Kong Group One              9
Hong Kong Group Two              8
Restricted Race                  6
Class 4 (Special Condition)      6
Class 3 (Special Condition)      2
Name: count, dtype: int64

In [72]:
race_df['race_name'].sample(5)

408                                          KUT CHEONG HANDICAP
2362                                         MIRACULOUS HANDICAP
1967                                          BUTTERFLY HANDICAP
642     THE SPORTS CLUB DIAMOND JUBILEE CHALLENGE CUP (HANDICAP)
1177                                            HEATHER HANDICAP
Name: race_name, dtype: object

In [54]:
race_df['race_name'].value_counts()

race_name
LYNDHURST HANDICAP                                     6
SATURN HANDICAP                                        6
KAM TIN RIVER HANDICAP                                 5
WONG NAI CHUNG HANDICAP                                5
DANDELION HANDICAP                                     5
                                                      ..
TONG FUK HANDICAP                                      1
AUDEMARS PIGUET HANDICAP                               1
THE PRINCE JEWELLERY & WATCH PREMIER CUP (HANDICAP)    1
VACHERON CONSTANTIN HANDICAP                           1
MEDIC KINGDOM HANDICAP                                 1
Name: count, Length: 1084, dtype: int64

In [76]:
race_df['race_course_track'].value_counts()

race_course_track
Sha Tin_ALL WEATHER TRACK           287
Happy Valley_TURF - "A" COURSE      264
Sha Tin_TURF - "A" COURSE           256
Sha Tin_TURF - "C+3" COURSE         251
Sha Tin_TURF - "C" COURSE           242
Happy Valley_TURF - "B" COURSE      236
Sha Tin_TURF - "B+2" COURSE         207
Happy Valley_TURF - "C" COURSE      202
Sha Tin_TURF - "A+3" COURSE         200
Happy Valley_TURF - "C+3" COURSE    152
Sha Tin_TURF - "B" COURSE            70
Name: count, dtype: int64

In [50]:
# sentiment analysis on hourse 

rid = np.random.randint(1, len(race_df))
print(race_df['incident_report'][rid])


                AUDACITY was crowded for room on jumping between TEAM SWEET and FOREVER ACCURATE which shifted in.
Shortly after the start, GAINFULJET and REGENCY BABY bumped, resulting in both horses becoming badly unbalanced.
From the outside barrier, SUPER SWEET ORANGE was shifted across behind runners in the early stages.
After the 1200 Metres, SUPER SWEET ORANGE got its head up on a number of occasions when proving difficult to settle.
After the 800 Metres, SMART SALUTE was left racing wide and without cover.
Near the 500 Metres, WORLD RECORD got its head on the side and shifted out towards the heels of SUPER SWEET ORANGE.
Shortly after entering the Straight, DIVINE DIYA was steadied when disappointed for running between TEAM SWEET and EQUITY DOCTRINE.
The Stewards inquired into the reason for Apprentice K K Chiong being dislodged from SUPER MAN passing the 350 Metres.  After taking evidence from Apprentice Chiong, in the presence of her allocated trainer, Mr K W Lui, N Callan (G